In [49]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import transformers
import pandas as pd
import datasets

modelpath = "./../models/checkpoint-4455/"

tokenizer = transformers.AutoTokenizer.from_pretrained(
    modelpath,

    model_max_length=100,
    padding_side="right",
    use_fast=True,
    trust_remote_code=True,
)

model = transformers.AutoModelForSequenceClassification.from_pretrained(
    modelpath,
    cache_dir=None,
    num_labels=1,
    trust_remote_code=True,
)

def tokenize_function(example):
        return tokenizer(example["seq"], truncation=True)

def post():
        dna = "Atggtgagcaagggcgaggagctgttcaccggggtggtgcccatcctggtcgagctggacggcgacgtaaacggccacaagttcagcgtgtccggcgagggcgagggcgatgccacctacggcaagctgaccctgaagttcatctgcaccaccggcaagctgcccgtgccctggcccaccctcgtgaccaccctgacctacggcgtgcagtgcttcagccgctaccccgaccacatgaagcagcacgacttcttcaagtccgccatgcccgaaggctacgtccaggagcgcaccatcttcttcaaggacgacggcaactacaagacccgcgccgaggtgaagttcgagggcgacaccctggtgaaccgcatcgagctgaagggcatcgacttcaaggaggacggcaacatcctggggcacaagctggagtacaactacaacagccacaacgtctatatcatggccgacaagcagaagaacggcatcaaggtgaacttcaagatccgccacaacatcgaggacggcagcgtgcagctcgccgaccactaccagcagaacacccccatcggcgacggccccgtgctgctgcccgacaaccactacctgagcacccagtccgccctgagcaaagaccccaacgagaagcgcgatcacatggtcctgctggagttcgtgaccgccgccgggatcactctcggcatggacgagctgtacaagtaa"
        dna = dna.upper()

        dna_length = len(dna)
        twenty_mers = []

        for i in range(dna_length - 19):
            twenty_mers.append(dna[i:i+20])

        df_twenty_mers = pd.DataFrame({'seq' : twenty_mers})
        raw_dataset = datasets.Dataset.from_pandas(df_twenty_mers)

        tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
        tokenized_dataset.set_format("torch")

        y_preds = []
        
        for row in tokenized_dataset:
            y_preds.append([row['seq'] ,model(row["input_ids"].unsqueeze(0), row["attention_mask"].unsqueeze(0))[0].item()])
            

        df_y_preds = pd.DataFrame(y_preds, columns=["seq", "Prediction"])
        df_y_preds.sort_values(by=['Prediction'], inplace=True, ascending=False)
        print(df_y_preds)



        

print(post())

C:\Users\robin/.cache\huggingface\modules\transformers_modules\bert_layers.py:125: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


Map:   0%|          | 0/701 [00:00<?, ? examples/s]

                      seq  Prediction
12   GGCGAGGAGCTGTTCACCGG    0.901075
55   TGGACGGCGACGTAAACGGC    0.864264
349  GCGACACCCTGGTGAACCGC    0.843734
54   CTGGACGGCGACGTAAACGG    0.842213
396  GAGGACGGCAACATCCTGGG    0.836199
..                    ...         ...
26   CACCGGGGTGGTGCCCATCC    0.067770
19   AGCTGTTCACCGGGGTGGTG    0.066918
157  AGCTGCCCGTGCCCTGGCCC    0.052020
373  AGCTGAAGGGCATCGACTTC    0.039140
28   CCGGGGTGGTGCCCATCCTG    0.007882

[701 rows x 2 columns]
None
